In [ ]:
import torch
from torch.autograd import Variable
import torchvision

import numpy as np
import os, time

from utilities.data import CIFAR10, MNIST, SVHN
from utilities.init import make_dirs, init_data_loader, write_preprocessor
from utilities.output import write_logger, write_observations
from utilities.parser import get_default_parser, update_code_dim
from architecture.nn import VAE

to_np = lambda x: x.data.cpu().numpy()
normalize_to_zero_one = lambda x: (x + 1.) / 2.
torch.cuda.set_device("cuda:4")

In [ ]:
torch.backends.cudnn.benchmark = True

parser = get_default_parser()
config = parser.parse_args(args=[])

config.img_size = 64
config.num_channels = 1
config.ckpt = True

update_code_dim(config)
write_preprocessor(config)

v = VAE(config)
v = v.cuda()

if config.ckpt:
    print("load_state")
    v.load_state_dict(torch.load(os.path.join(config.ckpt_path, 'v.pth')))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def generate_from_hidden(config, z, v, imgname):
    samples = v.Decoder(z)
    torchvision.utils.save_image(samples.data, imgname, normalize=True)
    return samples

In [ ]:
z_img = 'z.png'
x_img = 'generate.png'
z = Variable(torch.randn(config.num_samples, config.z_dim)).type(torch.cuda.FloatTensor)
gx = generate_from_hidden(config, z, v, x_img)
torchvision.utils.save_image(z.data, z_img)

In [ ]:
plt.imshow(mpimg.imread(z_img))

In [ ]:
plt.imshow(mpimg.imread(x_img))